In [1]:
import pandas as pd
test_df = pd.read_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/衡阳站多模型验证期结果.csv",parse_dates=["Date"])
test_df_copy = test_df
test_df_copy = test_df_copy.iloc[:, :2]
test_df.reset_index(drop=True)
test_df = test_df.set_index('Date')
"""
读取训练数据
"""
train_df = pd.read_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/衡阳站多模型率定期结果.csv",parse_dates=["Date"])
train_df_copy = train_df
train_df_copy = train_df_copy.iloc[:, :2]
train_df.reset_index(drop=True)
train_df = train_df.set_index('Date')
test_df

,Qobs,XAJ,TVGM,TANK,GR4J,BP,SVR,RNN,LSTM
Date,,,,,,,,,
1996-11-25,391,225.033998,0.000000,0.0000,735.3500,412.51212,452.107690,330.67706,359.58588
1996-11-26,383,254.349212,1.243738,222.5366,677.0830,427.12274,447.654346,335.34006,359.64227
1996-11-27,292,238.780517,9.174156,358.5957,500.3750,427.80844,427.297602,334.46700,353.75110
1996-11-28,316,215.054604,16.085507,232.6648,393.1510,357.89010,300.937053,258.40210,265.39352
1996-11-29,309,192.677854,37.273250,190.3943,322.3010,402.05334,401.121845,293.06906,333.27728
...,...,...,...,...,...,...,...,...,...
2001-12-27,766,303.171266,201.602220,632.9010,325.1101,705.56226,796.338839,617.83813,653.80597
2001-12-28,653,280.365424,164.880794,597.6152,317.9625,683.90850,816.775477,648.15950,669.88380
2001-12-29,536,260.064636,138.430022,561.9181,311.1167,565.22125,622.037698,570.21160,546.56840


In [2]:
# 导入进行线性回归的包

from sklearn.preprocessing import MinMaxScaler
# 导入随机森林包
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np
from tool import Indicator
"""
案例一：XAJ+GR4J
"""
# 处理数据
trainX_xaj = train_df.iloc[:, 1]
trainX_gr4j = train_df.iloc[:, 4]
trainy1 = train_df.iloc[:, :1]
trainx1 = pd.merge(trainX_xaj, trainX_gr4j, on="Date", how="left")
train_data1=pd.merge(trainy1,trainx1,on="Date",how="left")
testX_xaj = test_df.iloc[:, 1]
testX_gr4j = test_df.iloc[:, 4]
testy1 = test_df.iloc[:, :1]
testx1 = pd.merge(testX_xaj, testX_gr4j, on="Date", how="left")
test_data1=pd.merge(testy1,testx1,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled1 = scaler.fit_transform(train_data1)
df_for_testing_scaled1 = scaler.transform(test_data1)

# 划分数据
trainX1 = df_for_training_scaled1[:, 1:]
trainY1 = df_for_training_scaled1[:, :1]
testX1 = df_for_testing_scaled1[:, 1:]
testY1 = df_for_testing_scaled1[:, :1]


# 随机森林回归器
param_grid = {
    'n_estimators': [50, 100, 200],  #决策树的个数
    'max_depth': [3, 5, 7],  #最大树深，树太深会造成过拟合
}
rf = RandomForestRegressor()
# grid_model = KerasRegressor(build_fn=rf, verbose=1, validation_data=(testX, testY))
# grid_search = GridSearchCV(estimator=grid_model, param_grid=param_grid,
#                            cv=5)
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5)
grid_search = grid_search.fit(trainX1, trainY1)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict1 = model1.predict(trainX1)
train_predict1 = pd.DataFrame(train_predict1)
train_predict1.columns = ["RF1"]

test_predict1 = model1.predict(testX1)
test_predict1 = pd.DataFrame(test_predict1)
test_predict1.columns = ["RF1"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict1,len(test_data1.columns), axis=-1)
test_pred1 = scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict1),len(test_data1.columns))))[:, 0]
test_pred_copy1 = test_pred1
test_pred1 = pd.DataFrame(test_pred1)
test_pred1.columns = ["RF1"]

prediction_copies_array = np.repeat(train_predict1,len(test_data1.columns), axis=-1)
train_pred1 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict1),len(test_data1.columns))))[:, 0]
train_pred_copy1 = train_pred1
train_pred1=pd.DataFrame(train_pred1)
train_pred1.columns=['RF1']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred1['RF1'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred1['RF1'], test_df_copy['Qobs'])

/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vec

最佳参数组合：{'max_depth': 5, 'n_estimators': 100}
*******************率定期*******************
MAE:273.5334526061782
RMSE:456.58874771112863
NSE:0.8852820922462187
RE:-0.03883011754395747
Qmaxe:0.07766003898635483
Qmine:1.178293652748957
*******************验证期*******************
MAE:381.0128368607872
RMSE:628.9222376801821
NSE:0.8182212013041456
RE:7.6121275355344995
Qmaxe:0.10714443264315314
Qmine:1.6647087402560055


(381.0128368607872,
 628.9222376801821,
 0.8182212013041456,
 7.6121275355344995,
 0.10714443264315314,
 1.6647087402560055)

In [3]:
"""
案例二：LSTM+RNN
"""
# 处理数据
trainX_RNN = train_df.iloc[:, 8]
trainX_LSTM = train_df.iloc[:, 7]
trainy2 = train_df.iloc[:, :1]
trainx2 = pd.merge(trainX_LSTM, trainX_RNN, on="Date", how="left")
testX_RNN = test_df.iloc[:, 8]
testX_LSTM = test_df.iloc[:, 7]
testy2 = test_df.iloc[:, :1]
testx2 = pd.merge(testX_LSTM, testX_RNN, on="Date", how="left")
train_data2=pd.merge(trainy2,trainx2,on="Date",how="left")
test_data2=pd.merge(testy2,testx2,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled2 = scaler.fit_transform(train_data2)
df_for_testing_scaled2 = scaler.transform(test_data2)

# 划分数据
trainX2 = df_for_training_scaled2[:, 1:]
trainY2 = df_for_training_scaled2[:, :1]
testX2 = df_for_testing_scaled2[:, 1:]
testY2 = df_for_testing_scaled2[:, :1]


# 随机森林回归器
param_grid = {
    'n_estimators': [50, 100, 200],  #决策树的个数
    'max_depth': [3, 5, 7],  #最大树深，树太深会造成过拟合
}
rf = RandomForestRegressor()
# grid_model = KerasRegressor(build_fn=rf, verbose=1, validation_data=(testX, testY))
# grid_search = GridSearchCV(estimator=grid_model, param_grid=param_grid,
#                            cv=5)
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5)
grid_search = grid_search.fit(trainX2, trainY2)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict2 = model1.predict(trainX2)
train_predict2 = pd.DataFrame(train_predict2)
train_predict2.columns = ["RF2"]

test_predict2 = model1.predict(testX2)
test_predict2 = pd.DataFrame(test_predict2)
test_predict2.columns = ["RF2"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict2,len(test_data2.columns), axis=-1)
test_pred2 = scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict2),len(test_data2.columns))))[:, 0]
test_pred_copy2 = test_pred2
test_pred2 = pd.DataFrame(test_pred2)
test_pred2.columns = ["RF2"]

prediction_copies_array = np.repeat(train_predict2,len(test_data2.columns), axis=-1)
train_pred2 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict2),len(test_data2.columns))))[:, 0]
train_pred_copy2 = train_pred2
train_pred2=pd.DataFrame(train_pred2)
train_pred2.columns=['RF2']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred2['RF2'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred2['RF2'], test_df_copy['Qobs'])

/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vec

最佳参数组合：{'max_depth': 5, 'n_estimators': 200}
*******************率定期*******************
MAE:131.00230548442335
RMSE:249.7811985380079
NSE:0.9656679055084932
RE:-0.11104966428689471
Qmaxe:0.06720488721804536
Qmine:0.734613267197399
*******************验证期*******************
MAE:178.22630102768122
RMSE:336.2997247093832
NSE:0.9480240602031417
RE:-0.24004093012304026
Qmaxe:0.171302048726466
Qmine:1.121954093853129


(178.22630102768122,
 336.2997247093832,
 0.9480240602031417,
 -0.24004093012304026,
 0.171302048726466,
 1.121954093853129)

In [4]:
"""
案例三：LSTM+RNN+XAJ+GR4J
"""
# 处理数据
# 处理数据
trainy3 = train_df.iloc[:, :1]
trainx3 = pd.merge(trainx1, trainx2, on="Date", how="left")
testy3 = test_df.iloc[:, :1]
testx3 = pd.merge(testx1, testx2, on="Date", how="left")
train_data3=pd.merge(trainy3,trainx3,on="Date",how="left")
test_data3=pd.merge(testy3,testx3,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled3 = scaler.fit_transform(train_data3)
df_for_testing_scaled3 = scaler.transform(test_data3)

# 划分数据
trainX3 = df_for_training_scaled3[:, 1:]
trainY3 = df_for_training_scaled3[:, :1]
testX3 = df_for_testing_scaled3[:, 1:]
testY3 = df_for_testing_scaled3[:, :1]


# 随机森林回归器
param_grid = {
    'n_estimators': [50, 100, 200],  #决策树的个数
    'max_depth': [3, 5, 7],  #最大树深，树太深会造成过拟合
}
rf = RandomForestRegressor()
# grid_model = KerasRegressor(build_fn=rf, verbose=1, validation_data=(testX, testY))
# grid_search = GridSearchCV(estimator=grid_model, param_grid=param_grid,
#                            cv=5)
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5)
grid_search = grid_search.fit(trainX3, trainY3)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict3 = model1.predict(trainX3)
train_predict3 = pd.DataFrame(train_predict3)
train_predict3.columns = ["RF3"]

test_predict3 = model1.predict(testX3)
test_predict3 = pd.DataFrame(test_predict3)
test_predict3.columns = ["RF3"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict3,len(test_data3.columns), axis=-1)
test_pred3= scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict3),len(test_data3.columns))))[:, 0]
test_pred_copy3 = test_pred3
test_pred3 = pd.DataFrame(test_pred3)
test_pred3.columns = ["RF3"]

prediction_copies_array = np.repeat(train_predict3,len(test_data3.columns), axis=-1)
train_pred3 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict3),len(test_data3.columns))))[:, 0]
train_pred_copy3 = train_pred3
train_pred3=pd.DataFrame(train_pred3)
train_pred3.columns=['RF3']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred3['RF3'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred3['RF3'], test_df_copy['Qobs'])

/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/datou/anaconda3/envs/normal310/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vec

最佳参数组合：{'max_depth': 5, 'n_estimators': 200}
*******************率定期*******************
MAE:129.44173536772183
RMSE:243.10814515227338
NSE:0.9674778065789033
RE:-0.03894680955614164
Qmaxe:0.04673541492620494
Qmine:0.7350414281520635
*******************验证期*******************
MAE:178.5219796565764
RMSE:337.0917697856557
NSE:0.9477789469171172
RE:0.020497372659256625
Qmaxe:0.16922654402014772
Qmine:1.1224778635646602


(178.5219796565764,
 337.0917697856557,
 0.9477789469171172,
 0.020497372659256625,
 0.16922654402014772,
 1.1224778635646602)

In [5]:
test_df_full = pd.concat([test_df_copy, test_pred1, test_pred2, test_pred3], axis=1)
test_df_full = test_df_full.reset_index(drop=True)
test_df_full = test_df_full.set_index('Date')
test_df_full.to_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/HY_RF1Test.csv")
train_df_full = pd.concat([train_df_copy, train_pred1, train_pred2, train_pred3], axis=1)
train_df_full = train_df_full.reset_index(drop=True)
train_df_full = train_df_full.set_index('Date')
train_df_full.to_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/HY_RF1Train.csv")
test_df_full

,Qobs,RF1,RF2,RF3
Date,,,,
1996-11-25,391,600.724433,347.731765,345.659705
1996-11-26,383,604.814128,347.731765,345.659705
1996-11-27,292,583.972460,347.731765,345.659705
1996-11-28,316,489.684081,224.392107,225.216027
1996-11-29,309,469.866556,340.858410,343.340440
...,...,...,...,...
2001-12-27,766,506.255893,650.994017,649.872744
2001-12-28,653,464.180222,672.612784,671.384644
2001-12-29,536,458.133432,576.091934,579.211974
